In [101]:
import pandas as pd
import numpy as np
import re
data=pd.read_csv(r'C:\Users\bigdata\_Projet Neo4J\data.csv',sep=';')

In [102]:
#Sélection des colonnes

data=pd.DataFrame(data,columns=["ISBN","Langue","Titre","Titre de série","Editeur","Catégorie statistique 1","Catégorie statistique 2","Date",'Auteur Nom','Auteur Prénom',"Type de document",
 'Nombre de localisations',
 'Nombre de prêts 2017',
 "Nombre d'exemplaires"])

#Traitement des Titres
data['Titre 1']=data['Titre'].map(lambda x: x.split('|')[0])
data['Sous-titre']=data['Titre de série'].map(lambda x: " " if pd.isnull(x) else x)
data.drop('Titre de série', axis=1)

def func1(x):
    if len(x.split('|'))>1:
        result=x.split('|')[1]
    else:
        result=" "
    return result

data['Titre 2']=data['Titre'].map(func1)

In [103]:
#Je remove les types de document qui nous intéressent pas
typetodrop=['2,00',
 'Adaptation en Langue des Signes Française',
 'CD Rom revue',
 'Carte ou plan',
 'Cassette vidéo VHS > 12 ans',
 'Cassette vidéo VHS tous publics',
 'DVD jeunesse',
 'DVD nouveautés tous publics',
 'DVD- vidéo > 12 ans',
 'DVD- vidéo > 18 ans',
 'DVD-vidéo > 16 ans',
 'DVD-vidéo tous publics',
 'Diapositives jeunesse',
 'Disque compact',
 'Documents numériques et multimédia adulte',
 'Documents numériques et multimédia jeunesse',
 'Dossier',
 'Enregistrement musical pour la jeunesse',
 'Image, fiche cuisine …',
 'Instrument de musique',
 'Jeux Vidéos tous publics',
 'Jeux de société',
 'Jeux vidéos > 18 ans',
'Méthode de langue',
 'Méthode musicale',
 'Non empruntable',
 'Nouveauté',
 'Nouveauté disque compact',
 'Nouveauté jeunesse',
  'Livre sonore pour adulte',
 'Partition']
data = data[~data['Type de document'].isin(typetodrop)] #~permet de prendre not isin

In [104]:
#Traitement colonne auteur et Taux de prêt
data.isnull().sum()
data=data[~(data['Auteur Nom'].isnull()+data['Auteur Prénom'].isnull())]
data['Auteur']=data['Auteur Prénom']+" "+data['Auteur Nom']
data=data.drop(['Auteur Prénom','Auteur Nom'],axis=1)
data['Nombre de prêts par exemplaire']=data['Nombre de prêts 2017']/data['Nombre d\'exemplaires']

In [105]:
#Trier et ordonner colonnes
data=data.sort_values(['Nombre de prêts 2017'],ascending=False)
data=data[['Titre 1','Titre 2',"Sous-titre",'Auteur','Editeur','Date','Type de document','Nombre de prêts 2017','Nombre de prêts par exemplaire','Nombre d\'exemplaires','Nombre de localisations','Langue','Catégorie statistique 1']].set_index('Titre 1')

In [106]:
#Date
data['Date']=data["Date"].apply(lambda v: " " if  len(re.findall("(?:[^0-9]*)([0-9]{4})(?:[^0-9]*)",str(v)))==0 else re.findall("(?:[^0-9]*)([0-9]{4})(?:[^0-9]*)",str(v))[0])

In [107]:
Dict_cat2={'Bandes dessinees': ['LBDE Bandes dessinees'],
 'Histoire': ['D906 Histoire de Paris',
  'Histoire générale',
  'D902 Histoire ancienne',
  'Préhistoire',
  'D904 Histoire occidentale (Europe et Amerique du Nord)',
  'D504 Prehistoire',
  'CRUS Cinema russe et sovietique',
  'D903 Histoire de France',
  'D003 Encyclopedies',
  'D905 Histoire: autre pays',
  'D901 Histoire generale'],
 'Langues': ['D410 Langues: autres langues',
  'D405 Langues: italien',
  'D408 Langues: arabe',
  'D401 Langues et linguistique',
  'D407 Langues: portugais',
  'D402 Langues: anglais, americain',
  'D403 Langues: allemand',
  'D409 Langues: chinois, vietnamien, cambodgien',
  'D406 Langues: espagnol',
  'D404 Langues: francais, FLE',
  'Langues et langage'],
 'N/A': ['5',
  '135',
  '61',
  '64',
  'D001 Generalites (autres)',
  '62',
  'LCOM',
  '66',
  'INST',
  '264',
  '4',
  'MASO',
  '265',
  '0',
  '9',
  '2',
  'GSOC',
  '262',
  '137',
  '138',
  '3',
  '7',
  '268',
  'M402',
  '140',
  '153',
  '155',
  '151',
  '1',
  '141',
  'Généralités (autres)',
  '269',
  'CSPO Captation: autres (sport…)',
  '261',
  '65'],
 'Musique': ['MMIN Methodes musicales: instruments',
  'M102 Negro spirituals, gospel',
  'M904 Musiques du monde: monde arabe',
  'M802 Chanson de varietes',
  "M201 Rock'n'roll, rockabilly",
  'M912 Musiques du monde: Amerique du Nord',
  'M914 Musiques du monde: Bresil',
  'M112 Hip hop, rap',
  'M114 Reggae',
  'M110 Jazz variete et jazz "inclassable"',
  'M501 Musiques fonctionnelles',
  'M310 Musique classique: 20eme et 21eme siecles',
  "M307 Musique classique: epoque baroque (jusqu'a 1750)",
  'M908 Musiques du monde: peninsule iberique',
  'MMSO Methodes musicales: solfege',
  'M902 Musiques du monde: peuples en diaspora',
  'M701 Musiques inclassables',
  'M906 Musiques du monde: Extreme-Orient et Pacifique',
  "M907 Musiques du monde: Europe de l'Est et méridionale",
  'M803 Rock francais',
  'M105 Be-bop',
  'M401 Musiques electroniques',
  'M304 Musique classique: musique lyrique (opera, operettes)',
  'M303 Musique classique: recital chant',
  'M909 Musiques du monde: France',
  'M106 Cool jazz',
  'LALB Albums',
  'M101 Blues',
  'M309 Musique classique: epoque romantique et post-romantique',
  'M103 Jazz preclassique',
  'M901 Musiques du monde: anthologies internationales',
  'M302 Musique classique: recital instruments',
  'M202 Pop',
  'M209 Fusion de styles, rock "inclassable"',
  'M911 Musiques du monde: musiques celtiques',
  'M108 Jazz fusion',
  "M111 Rythm'n'blues, soul",
  'M305 Musique classique: Moyen Age',
  'M306 Musique classique: Renaissance',
  'M104 Jazz classique',
  'M207 New wave, cold wave',
  'M206 Punk',
  'M301 Musique classique: anthologies thematiques',
  'M915 Musiques du monde: Antilles hispanophones et francophones',
  'M107 Free jazz',
  'M601 Musiques de films et compilations',
  'M308 Musique classique: epoque classique',
  'M913 Musiques du monde: Amerique latine sauf Bresil',
  'M905 Musiques du monde: Asie',
  'M208 Rock et varietes rock',
  'M203 Folk rock, country rock, blues rock',
  'M205 Hard rock, metal',
  'M204 Rock psychedelique, progressif…',
  'D707 Musique',
  'M109 Jazz-rock',
  "M910 Musiques du monde: Europe de l'Ouest et du Nord",
  'M903 Musiques du monde: Afrique'],
 'Actualité': ["D006 Periodiques: hebdomadaires d'actualite",
  'D005 Periodiques: quotidiens',
  'D004 Presse'],
 'Sciences': ['D601 Sciences appliquees generalites, metrologie',
  'Zoologie',
  'D507 Zoologie',
  'Médecine sauf sexualité',
  'D502 Math., astronomie, physique, chimie',
  'D501 Sciences generalites',
  'D102 Pseudo sciences',
  'D505 Sciences de la vie',
  'D602 Medecine et sexualite',
  'Sciences généralités',
  'D503 Sciences de la terre sauf prehistoire',
  'Math., astronomie, physique, chimie'],
 'Litterature': ['LNAM Litterature nord-americaine',
  'LNOR Litterature nordique',
  'LESP Litterature espagnole',
  'D802 Anthologies litteraires',
  'LMMO Litterature du Maghreb et du Moyen-Orient',
  'Littérature française (autres)',
  'LJAP Litterature japonaise et coreenne',
  'LBRI Litterature britannique',
  'LGER Litterature germanique',
  'LAFR Litterature africaine sub-saharienne',
  'Poésie française',
  'LIND Litterature indienne et asiatique',
  'Etudes littéraires, revues littéraires',
  'LCHI Litterature chinoise',
  'D801 Etudes litteraires, revues litteraires',
  "LECE Litterature d'Europe centrale",
  'Fiction en langue etrangere',
  'LRUS Litterature russe et sovietique',
  'LAUS Litterature australienne et du Pacifique',
  'Littérature étrangère',
  'LPOR Litterature portugaise',
  'Romans en langue francaise',
  'LBAL Litterature des Balkans',
  'LLAT Litterature latino-americaine',
  'LANT Litteratures antiques',
  'LFRA Litterature francaise'],
 'Architecture': ['Urbanisme, architecture',
  'LITA Litterature italienne',
  'M801 Chanson a texte',
  'D702 Urbanisme, architecture'],
 'Enfance': ['Pédagogie',
  'D306 Pedagogie',
  'MJCR Phonos enfants\xa0: chansons et ronds',
  'MJEM Phonos enfants\xa0: eveil sonore et musical'],
 'Philosophie': ['D105 Oeuvres philosophiques: modernes',
  'Philosophie',
  'D101 Philosophie',
  'Grands textes philosophiques',
  'D104 Oeuvres philosophiques: antiquite'],
 'Cinéma': ['CITA Cinema italien',
  'CCHI Cinema chinois',
  'CAFR Cinema africain sub-saharien',
  'CFRA Cinema francais',
  'CJAP Cinema japonais et coreen',
  'CGER Cinema germanique',
  'CIND Cinema indien et asiatique',
  'CBRI Cinema britannique',
  'Cinéma',
  'CAUS Cinema australien et du Pacifique',
  'CNAM Cinema nord-americain',
  'CLAT Cinema latino-americain',
  'CBAL Cinema des Balkans',
  "CECE Cinema d'Europe centrale",
  'CNOR Cinema nordique',
  'CMMO Cinema du Maghreb et du Moyen-Orient',
  'D708 Cinema',
  'CPOR Cinema portugais'],
 'Sciences sociales': ['D301 Sciences sociales (autres)',
  'Sciences sociales (autres)',
  'Psychologie',
  'D103 Psychologie'],
 'Culture': ['D712 Autres spectacles, radio, television, loisirs et jeux',
  'D307 Coutumes, ethnologie, folklore',
  'CHUM Captation: humour',
  'Coutumes, ethnologie, folklore',
  'D911 Guides de voyage, tourisme',
  'Autres spectacles, radio, télévision, loisirs et jeux',
  'CSPE Captation: autres spectacles'],
 'Environnement': ['D308 Ecologie, developpement durable'],
 'Religion': ['D204 Autres religions',
  'D203 Judaisme, islam',
  'Religions non chrétiennes',
  'Religion (autres)',
  'D201 Religions: generalites',
  'Bible, christianisme',
  'D202 Bible, christianisme'],
 'Droit': ['D305 Droit, administration, questions sociales',
  'Droit, administration, questions sociales'],
 'Art': ['D701 Arts generalites',
  'Peinture, arts graphiques',
  'D710 Danse',
  'D705 Peinture, arts graphiques',
  'D703 Sculpture',
  'Photographie',
  'Arts décoratifs',
  'D506 Botanique',
  'Sculpture',
  'D704 Arts decoratifs',
  'Arts généralités',
  'D706 Photographie'],
 'Mangas': ['LMAN Mangas'],
 'Industrie & Technologie': ['D608 Industrie, artisanat, batiments',
  'D604 Elevage, horticulture'],
 'Informatique': ['D603 Techniques sauf informatique',
  'D002 Informatique et ordinateurs'],
 'Vie pratique': ['D606 Cuisine, alimentation',
  'D605 Vie pratique sauf cuisine',
  'Vie pratique sauf cuisine',
  'Cuisine, alimentation'],
 'Politique': ['D302 Politique', 'Politique'],
 'Geographie': ['D910 Geographie'],
 'Metiers': ['D304 Metiers'],
 'Economie': ['D607 Gestion',
  'Gestion',
  'D303 Economie sauf metiers, commerce',
  'Economie sauf métiers, commerce'],
 'Theatre': ['Théâtre français', 'D709 Theatre'],
 'Sport': ['D711 Sport', 'Sport']}
Dictio={}
for x,y in Dict_cat2.items():
    for cat in y:
        Dictio[cat]=x

In [108]:
#Map Categories
data['Catégorie']=data["Catégorie statistique 1"].map(Dictio)

In [109]:
data=data.reset_index().rename(columns={"Titre 1": "Titre", "Nombre de prêts 2017": "prets","Nombre d'exemplaires":"exemplaires","Nombre de localisations":"localisations","Catégorie statistique 1":"cat","Sous-titre":"sstitre"})

In [110]:
maxpret=max(data["prets"]/data["exemplaires"])
data["tauxemprunt"]=round((1-(maxpret-data["prets"]/data["exemplaires"])/maxpret)*100,0)

In [111]:
data.to_csv(r'C:\Users\bigdata\_Projet Neo4J\data_treated.csv')

In [112]:
#Que litterature
data_litterature = data[data['Type de document']!="Livre jeunesse"]
data_litterature=data_litterature[data_litterature["Catégorie"]=="Litterature"].dropna().drop(["Type de document","Titre 2","Catégorie"],axis=1)
data_litterature["Titre"]=data_litterature["Titre"].map(lambda x: re.sub('\"',' ',str(x)))
data_litterature["sstitre"]=data_litterature["sstitre"].map(lambda x: re.sub('\"',' ',str(x)))
data_litterature["Auteur"]=data_litterature["Auteur"].map(lambda x: re.sub('\"',' ',str(x)))
data_litterature["Editeur"]=data_litterature["Editeur"].map(lambda x: re.sub('\"',' ',str(x)))


In [118]:
data_litterature.to_csv(r'C:/Users/bigdata/Neo4J-Project/data_litterature.csv',encoding="UTF-8")
data_litterature.to_csv(r'C:\Users\bigdata\_Projet Neo4J\neo4jDatabases\database-fe30ce40-e8ac-4fa5-84a4-58d31567adc7\installation-3.5.6\import/data_litterature.csv',encoding="UTF-8")

In [114]:
data_litterature.head()

,Titre,sstitre,Auteur,Editeur,Date,prets,Nombre de prêts par exemplaire,exemplaires,localisations,Langue,cat,tauxemprunt
24,Chanson douce,,Leïla Slimani,Gallimard,2016,1169.0,16.236111,72.0,51.0,français,LFRA Litterature francaise,68.0
137,Petit pays,,Gaël Faye,Bernard Grasset,2016,889.0,14.338710,62.0,47.0,français,LFRA Litterature francaise,60.0
149,Sur les chemins noirs,,Sylvain Tesson,Gallimard,2016,856.0,16.150943,53.0,49.0,français,LFRA Litterature francaise,67.0
151,Arrête avec tes mensonges,,Philippe Besson,Julliard,2017,846.0,16.588235,51.0,49.0,français,LFRA Litterature francaise,69.0
184,Article 353 du code pénal,,Tanguy Viel,les Éditions de Minuit,2017,785.0,14.811321,53.0,52.0,français,LFRA Litterature francaise,62.0


In [40]:
#Programme pour recatégoriser une série

def Categorize(serie):
    categories=set(serie.tolist())
    Dict_cat={i:j for (i,j) in enumerate(categories)}
    Dict_cat2={}
    List_cat2=[]
    for i in Dict_cat:
        print("\n",Dict_cat[i])
        catnew=input("newcat:")
        if catnew!="n":
            List_cat2.append(catnew)
            List_cat3={i:j for (i,j) in enumerate(List_cat2)}
            Dict_cat2[Dict_cat[i]]=catnew
        else:
            print(List_cat3)
            index=int(input("indexcat:"))
            catold=List_cat3[index]
            Dict_cat2[Dict_cat[i]]=catold
        print(List_cat3)
    return Dict_cat2
